In [2]:
import os

# setup stuff.  The cell just supports the workbook, you can ignore it


DO_FOLDER = os.path.abspath(f"{os.getcwd()}/../do/hello/cube_examples")
FILE_CONTENTS = '''
hello_world.py
def hello_world():
    print("   hello world!")
---
hello_again.py
def hello_again():
    print("   Hello World again!")


def hella():
    print("   HELLA Hello World!!!  Hello world again.")


def salutation(name="Hello", *, emphasis=False, lucky_number=999):
    line = F"   {name}, My lucky number is {lucky_number}"
    print(F"{line.upper()}!" if emphasis else line)
    return lucky_number
'''

def add_loadable(name, *, show=True):
    at, contents = FILES[name]
    path = os.path.join(DO_FOLDER, at)
    if not os.path.exists(p := os.path.dirname(path)):
        os.makedirs(p)
    with open(path, 'w') as f:
        f.write(contents)
    with_bar = "\n  | ".join(contents.split("\n"))
    if show:
        print(F"\n### ADDED LOADABLE {'do/'+at!r}\n  | {with_bar}\n\n")

def add_all_loadables():
    for name in FILES.keys():
        add_loadable(name, show=False)
        
def cleanup_loadables():
    for suffix in FILES.keys():
        path = os.path.join(DO_FOLDER, at)
        if os.path.exists(path):
            os.remove(path)

def run_do(*args, **kwargs):
    parts = [repr(x) for x in args] + [F"{k}={v!r}" for k, v in kwargs.items()]
    print(F"do({', '.join(parts)})")
    result = do(*args, **kwargs)
    print(F"--> {result!r}")
    print()


FILES = {}
for f in map(str.strip, FILE_CONTENTS.split('---')):
    lines = f.split('\n')  
    FILES[lines[0]] = (lines[0], '\n'.join(lines[1:]))
    add_loadable(lines[0], show=False)
    

# add_all_loadables()
from utils.do import do, load, _DO_FOLDER  # Add all loadables BEFORE loading this module
print(F"The loadables are added to the {DO_FOLDER!r} folder")
print(F"Do module's do folder: {_DO_FOLDER!r}")
if DO_FOLDER != _DO_FOLDER:
    print("WARNING: DO_folder for jupyter = {DO_FOLDER} does not match {_DO_FOLDER}")
if not os.path.exists(DO_FOLDER):
    input(f"WARNING: Do folder {DO_FOLDER} not found.")
    os.makedirs(DO_FOLDER)
print("\n\n\n\n")




DO_FOLDER = /Users/oblinger/ob/proj/sv/algorithms2/do
The loadables are added to the '/Users/oblinger/ob/proj/sv/algorithms2/do/hello' folder
Do module's do folder: '/Users/oblinger/ob/proj/sv/algorithms2/do'







In [3]:
add_loadable("hello_world.py")


### ADDED LOADABLE 'do/hello_world.py'
  | def hello_world():
  |     print("   hello world!")




In [4]:
from utils.do import _DO_FOLDER
_DO_FOLDER

'/Users/oblinger/ob/proj/sv/algorithms2/do'

In [5]:
do("hello_world")

   hello world!


.
## EXAMPLE -- MULTIPLE DO FUNCTIONS DEFINED IN ONE MODULE
One can put multiple do functions in one file and reference them with a dot notion as shown here.

In [6]:
add_loadable("hello_again.py")


### ADDED LOADABLE 'do/hello_again.py'
  | def hello_again():
  |     print("   Hello World again!")
  | 
  | 
  | def hella():
  |     print("   HELLA Hello World!!!  Hello world again.")
  | 
  | 
  | def salutation(name="Hello", *, emphasis=False, lucky_number=999):
  |     line = F"   {name}, My lucky number is {lucky_number}"
  |     print(F"{line.upper()}!" if emphasis else line)
  |     return lucky_number




In [7]:
do("hello_again.hella")

   HELLA Hello World!!!  Hello world again.


.
## EXAMPLE -- PASSING ARGS AND RESULTS
Here we see fixed and keyword args being forwarded by do to the underlying function.
And likewise its result is forward back to be the result of the do call.

In [8]:
do("hello_again.salutation", "Michael", emphasis=True)

   MICHAEL, MY LUCKY NUMBER IS 999!


999

.
## EXAMPLE -- ALL SUB-FOLDERS ARE SCANNED FOR "DO" FUNCTION
Here we see "deep_hello" is called even when it occurs deeply within the folder tree.

In [9]:
add_loadable("deep/deep/deep/deep_hello.py")


### ADDED LOADABLE 'do/deep/deep/deep/deep_hello.py'
  | def deep_hello():
  |     print("hello echoing out from deep in the filesystem!")




In [10]:
do("deep_hello")

hello echoing out from deep in the filesystem!


.
.
# >>> CALLING A CONFIGURATION <<<
In addition to invoking a simple function, "do" can also invoke a configuration dict.
In this case:
1. The dict is expanded by recursively looking up "main.base" and using its values tree as defaults
2. Then finally calling the function associated with "main.do"
3. The expanded dict is passed as the first arg followed by args passed to do

.
## EXAMPLE -- CALLING A CONFIG
Here 'hello_config' loads a json file instead of a python function.
In this case the "main.do" value of "hello config action" is loaded and called.

In [11]:
add_loadable("hello_config.json")


### ADDED LOADABLE 'do/hello_config.json'
  | {
  |     "main": {
  |         "do": "configurable_salutation"
  |     },
  |     "name": "Hello",
  |     "lucky_number": 7,
  |     "emphasis": false
  | }




In [12]:
add_loadable("configurable_salutation.py")


### ADDED LOADABLE 'do/configurable_salutation.py'
  | def configurable_salutation(spec, name=None, *, emphasis=False, lucky_number=None):
  |     name = spec.get("name") if name is None else name
  |     emphasis = spec.get("emphasis")
  |     lucky_number = spec.get("lucky_number")
  |     line = F"   {name}, My lucky number is {lucky_number}"
  |     print(F"{line.upper()}!" if emphasis else line)
  |     return lucky_number




In [13]:
do("hello_config", "Martin")

   Martin, My lucky number is 7


7

.
## EXAMPLE -- CONFIG INHERITANCE
Here 'hello_shadowing_config' sets lucky_number to 777 and inherits function to call and other parameters from 'hello_config'.

In [14]:
add_loadable("hello_shadowed_config.json")


### ADDED LOADABLE 'do/hello_shadowed_config.json'
  | {
  |   "main": {
  |     "base": "hello_config" },
  |   "lucky_number":  777
  | }




In [15]:
do("hello_shadowed_config")

   Hello, My lucky number is 777


777

_
## EXAMPLE -- COMBINING CONFIGS AND CODE
Complex tools (including nearly a visualizers/report generators) naturally have simple config info best expressed as a config dict,
and complex config best expressed in python.  Forcing these to be separate loadables will generate confusing sea of many tiny 
separate 2-line loadable files.

To address this "do" allows config data (normally stored in .json) to be stored in a variable in a .py file.  This allows that
config info to be bundled with functions that are referenced by that same config in the same module.  

The example below shows a silly complex tool that applies a sequence to text transformation rules to a sequence of letters.
The first loadable provides a config with the base parameters and the rule engine itself.  The second loadable configures the tool and 
provides a couple of small python rule functions that are used by the configuration all nicely wrapped up in a single .py file.

In [16]:
add_loadable("letterator.py")


### ADDED LOADABLE 'do/letterator.py'
  | from utils.do import load
  | 
  | """Silly configurable tool for applying rules to a sequence of letters."""
  | letterator = {
  |     "main": {
  |       "do": "letterator.run",      # example of a complex tool config
  |       "title": "The Letterator"
  |     },   
  |     "start": 48,
  |     "end": 122
  | }
  | 
  | def run(spec):
  |     results = []
  |     for idx in range(spec["start"], spec["end"]):
  |         text = chr(idx)
  |         for step, rule_name in spec["rules"]:
  |             fn = load(rule_name)
  |             if idx % step == 0:
  |                 text = fn(idx, text)
  |         results.append(text)
  |     print(spec["main"]["title"])
  |     return "  ".join(results)




In [17]:
add_loadable("my_letters.py")


### ADDED LOADABLE 'do/my_letters.py'
  | my_letters = {
  |   "main": {"base": "letterator"},
  |   "start": 97,
  |   "rules": [
  |     (7, "my_letters.jackpot"),
  |     (3, "my_letters.triple_it"),
  |     (5, "my_letters.all_caps_it")]
  | }
  | 
  | def triple_it(idx, text):
  |     return F"{text}{text}{text}"
  | 
  | def all_caps_it(idx, text):
  |     return text.upper()
  | 
  | def jackpot(idx, text):
  |     return "jackpot "




In [18]:
do("my_letters")

The Letterator


'a  jackpot   ccc  D  e  fff  g  h  JACKPOT JACKPOT JACKPOT   j  k  lll  m  N  ooo  jackpot   q  rrr  S  t  uuu  v  jackpot   XXX  y'

.
.
# USE CASE - SELF DOCUMENTING PROCESSES
When possible we can use simple verisoned object to help us execute coding processes, and 
track/maintain those processes.  

.
## EXAMPLE -- Loadable constant
Here we show that a loadable can be any python constant data value.
In this example we have a set of named lists that are used to track 
our supported dataset, metrics, and tools.

This versioned data structure is used as input by the 'naughtly_list' script that scans
supported components to see that each has (1) a doc string, both quick and full regression tests, etc.


In [19]:
add_loadable("supported.yaml")


### ADDED LOADABLE 'do/supported.yaml'
  | datasets:
  | - regression_games # Any game referenced by any regression test MUST be listed here
  | - baller10  # Default dataset use by all basketball metrics
  | - volley10
  | - arron4    # Examples of higher resolution games
  | metrics:
  | - team_highlight.money  # Jason agreed money metric for team highlights 
  | - team_highlight.precision # Just the precision portion of this metric
  | - player_highlight.money # Jason agreed, include player ID
  | - basket_stats.money # Jason agree, metric for points, player, make-miss stats
  | - p_metric # used in 2022




In [20]:
add_loadable("team_highlight.py")


### ADDED LOADABLE 'do/team_highlight.py'
  | """
  | Team highlight money is the F1 where correctness is tied to correctly assessing shot 
  | attempt and make-miss, without consideration of player-ID nor number of points scored.
  | 
  | This is the Jason approved metric associated with our team highlights product, and we
  | have agreed 80% is the minimum approved threshold required for product ship.
  | """
  | 
  | from inst import Inst
  | 
  | def reg_quick_test():
  |     run_result = Inst.load("reg1_latest")   # Reg1 is the pickle for a special 5-min game snipit
  |     assert do("team_highlight_money", run_result) > .65
  | 
  | reg_full_test = "std_full1"  # indicates full regression testing is part of 'std_full1'
  | 
  | def money(run_result: Inst) -> float:
  |     return -1 # implementation goes here
  | 
  |     
  | def precision(run_result: Inst) -> float:
  |     return -1 # implementation goes here




In [21]:
add_loadable("naughty_list.py")


### ADDED LOADABLE 'do/naughty_list.py'
  | """
  | The "naughty list" scans all supported datasets, metrics, visualization/debugging tools 
  | and verifies they are (1) properly documented, (2) they execute their full regressions, 
  | (3) The continue to run against representative games, metrics, tools.
  | 
  | Any metric, tool, dataset that is not fully compliant is indicated on the naugthy list.
  | """
  | 
  | from do import load
  | 
  | def naughty_list():
  |     # this double for loop checks docs exist, regression test exists, and passes etc.
  |     for section, supported_insts in load("supported").items():
  |         for name in supported_insts:
  |             try:
  |                 inst = load(name)
  |                 if not hasattr(inst, "__DOC__"):
  |                     print(F"   {section} {name} does not have a valid doc string")
  |                 if not hasattr(inst, ):
  |                     print(F"   {section} {name} does not have a valid doc string")


In [22]:
# Note this code cannot run since
from inst import Inst
from do import load

reg1_name = load("supported")["datasets"][0]   # Gets then name of a mcproc result to use
reg1 = Inst(spec={}, path=".")     # This should be Inst.load(reg1) but that inst does not exist here
score = do("team_highlight.money", reg1)  # computes money metric on reg1

do("naughty_list")   # runs our checking code

ModuleNotFoundError: No module named 'src'

.
.
# USING DO FROM THE COMMAND LINE
Do encapsulates execution as a self describing building block.  The do function is designed to be easily 
embedded within larger execution scripts.  In some cases it is convenient for a user to directly invoke do
as a toplevel command.  The do commandline interface provide command line support "for free" for any such 
do function. It defines a simple mapping from expected --arguments and -a argument onto Python fixed and kwargs.
This probably best shown using a series of examples:

.
### EXAMPLE -- Showing default usage command for 

In [23]:
!./do --usage

zsh:1: no such file or directory: ./do


.
### EXAMPLE -- INVOKING A DO FUNCTION FROM THE COMMAND LINE
Earlier we had hello salutation that took fixed and keyword args.
Without additonal configuration we can invoke it from the command line
using UNIX style args and flags as shown here:

In [24]:
!../do hello_again.salutation Maxim --emphasis

zsh:1: no such file or directory: ./do


.
### EXAMPLE -- INVOKING A CONFIGURED TOOL FROM THE COMMAND LINE
In this example we show one also can invoke a do configuration from the command line as well.
Here we have the same configurable "letterator" tool invoked as a do function above:

In [38]:
add_loadable("my_letters.py")


### ADDED LOADABLE 'do/my_letters.py'
  | my_letters = {
  |   "main": {"base": "letterator"},
  |   "start": 97,
  |   "rules": [
  |     (7, "my_letters.jackpot"),
  |     (3, "my_letters.triple_it"),
  |     (5, "my_letters.all_caps_it")]
  | }
  | 
  | def triple_it(idx, text):
  |     return F"{text}{text}{text}"
  | 
  | def all_caps_it(idx, text):
  |     return text.upper()
  | 
  | def jackpot(idx, text):
  |     return "jackpot "




In [63]:
!./do my_letters

The Letterator
a  jackpot   ccc  D  e  fff  g  h  JACKPOT JACKPOT JACKPOT   j  k  lll  m  N  ooo  jackpot   q  rrr  S  t  uuu  v  jackpot   XXX  y


### EXAMPLE -- TWEAK CONFIG FROM COMMANDLINE
Often we script and configure a complex test, but then we want to tweak one or two parameters over and over and check our results.
(This becomes especially powerful when intermediate results are cached, so retesting is fast.)

In [62]:
!./do my_letters --set main.title "Re-configured letterator" --json rules '[[2, "my_letters.triple_it"]]'

Re-configured letterator
a  bbb  c  ddd  e  fff  g  hhh  i  jjj  k  lll  m  nnn  o  ppp  q  rrr  s  ttt  u  vvv  w  xxx  y


.
### EXAMPLE -- SETTING MULTIPLE PARAMETERS AT ONCE
The --sets keyword can perform multiple simple assigments at once

In [67]:
!./do my_letters --sets main.title=Quickie,start=100,end=110

Quickie
D  e  fff  g  h  JACKPOT JACKPOT JACKPOT   j  k  lll  m


# CLEANUP
Removes loadable files created by this example script

In [ ]:
cleanup_loadables()